<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/WOOK/attention_Mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install accelerate -U
!pip install transformers[torch] -U
!pip install shap
!pip install datasets
!pip install seaborn
!pip install matplotlib
!pip install ipython


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 1.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
##### 취약한 코드라인 식별 #####
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from IPython.core.display import display, HTML
import re

# 모델과 토크나이저 로드
model_name = 'microsoft/codebert-base-mlm'
model_path = '/content/drive/MyDrive/Colab Notebooks/model/codebert(func)'
model = AutoModelForSequenceClassification.from_pretrained(model_path, output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_code(code):
    code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)  # 멀티라인 주석 제거
    code = re.sub(r'//.*?\n', '', code)  # 싱글라인 주석 제거
    code = re.sub(r'#include .*?\n', '', code)  # include 문 제거
    code = re.sub(r'#include ".*?\n', '', code)  # include 문 제거
    code = re.sub(r'#define .*?\n', '', code)  # 매크로 정의 제거
    code = re.sub(r'#undef\s+\w+', '', code)  # 전처리 지시문 제거
    code = re.sub(r'#if\s+\w+', '', code)  # 전처리 지시문 제거
    code = re.sub(r'#else\s+\w+', '', code)  # 전처리 지시문 제거
    code = re.sub(r'#elif\s+\w+', '', code)  # 전처리 지시문 제거
    code = re.sub(r'#endif', '', code)  # 전처리 지시문 제거
    code = re.sub(r'[\t ]+', ' ', code)  # 탭과 여러 공백을 하나의 공백으로
    code = re.sub(r'\n\s*\n', '\n', code)  # 여러 줄바꿈을 하나로
    code = re.sub(r'\n', ' ', code)  # 줄바꿈 제거
    code = re.sub(r'return*.*?;', '', code)  # return 문 제거
    code = re.sub(r'return;', '', code)  # return 문 제거
    return code.strip()

def calculate_attention_per_line(model, tokenizer, code_lines):
    line_attention_scores = []
    for line in code_lines:
        inputs = tokenizer(line, return_tensors='pt', truncation=True, max_length=512, padding=True)
        input_ids = inputs['input_ids']
        attention = model(input_ids, output_attentions=True).attentions
        avg_attn = np.mean([att.detach().cpu().numpy() for att in attention], axis=(0, 1, 2)).squeeze()
        line_attention_scores.append(np.mean(avg_attn))
    return line_attention_scores

def highlight_code_lines(code_lines, attention_scores, threshold=0.01):
    max_attn = max(attention_scores)
    min_attn = min(attention_scores)
    html_code = "<pre>"
    for line, score in zip(code_lines, attention_scores):
        if score > threshold:
            color_intensity = int(255 * (score - min_attn) / (max_attn - min_attn))
            color = f'rgba(255,{255-color_intensity},{255-color_intensity},0.5)'
            html_code += f'<span style="background-color:{color}">{line}</span>\n'
        else:
            html_code += f'{line}\n'
    html_code += "</pre>"
    display(HTML(html_code))

# 코드 예시
code_example = '''
void func(YR_SCAN_CONTEXT * context, YR_AC_MATCH * ac_match, uint8_t * data, size_t data_size, size_t data_base, size_t offset) {
  CALLBACK_ARGS callback_args;
  RE_EXEC_FUNC exec;
  int forward_matches = -1;
  int backward_matches = -1;
  int flags = 0;
  if (STRING_IS_GREEDY_REGEXP(ac_match -> string)) flags |= RE_FLAGS_GREEDY;
  if (STRING_IS_NO_CASE(ac_match -> string)) flags |= RE_FLAGS_NO_CASE;
  if (STRING_IS_DOT_ALL(ac_match -> string)) flags |= RE_FLAGS_DOT_ALL;
  if (STRING_IS_FAST_REGEXP(ac_match -> string)) exec = yr_re_fast_exec;
  else exec = yr_re_exec;
  if (STRING_IS_ASCII(ac_match -> string)) {
    forward_matches = exec(ac_match -> forward_code, data + offset, data_size - offset, offset > 0 ? flags | RE_FLAGS_NOT_AT_START : flags, NULL, NULL);
  }
  if (STRING_IS_WIDE(ac_match -> string) && forward_matches == -1) {
    flags |= RE_FLAGS_WIDE;
    forward_matches = exec(ac_match -> forward_code, data + offset, data_size - offset, offset > 0 ? flags | RE_FLAGS_NOT_AT_START : flags, NULL, NULL);
  }
  switch (forward_matches) {
  case -1:
  case -2:
  case -3:
  case -4:
  case -5:
  }
  if (forward_matches == 0 && ac_match -> backward_code == NULL) callback_args.string = ac_match -> string;
  callback_args.context = context;
  callback_args.data = data;
  callback_args.data_size = data_size;
  callback_args.data_base = data_base;
  callback_args.forward_matches = forward_matches;
  callback_args.full_word = STRING_IS_FULL_WORD(ac_match -> string);
  if (ac_match -> backward_code != NULL) {
    backward_matches = exec(ac_match -> backward_code, data + offset, offset, flags | RE_FLAGS_BACKWARDS | RE_FLAGS_EXHAUSTIVE, _yr_scan_match_callback, (void * ) & callback_args);
    switch (backward_matches) {
    case -2:
    case -3:
    case -4:
    case -5:
    }
  } else {
    FAIL_ON_ERROR(_yr_scan_match_callback(data + offset, 0, flags, & callback_args));
  }
}
'''

# 코드 전처리
preprocessed_code = preprocess_code(code_example)

# 코드 라인을 분할
code_lines = preprocessed_code.strip().split(';')

# 각 라인의 어텐션 가중치 계산
attention_scores = calculate_attention_per_line(model, tokenizer, code_lines)

# 취약한 라인 강조
highlight_code_lines(code_lines, attention_scores, threshold=0.01)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
##### 어떤 CWE에 해당하는 코드인지 #####
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# 새 코드 데이터 준비
new_code_data = {
    'code': [
        """
        void func(YR_SCAN_CONTEXT * context, YR_AC_MATCH * ac_match, uint8_t * data, size_t data_size, size_t data_base, size_t offset) {
  CALLBACK_ARGS callback_args;
  RE_EXEC_FUNC exec;
  int forward_matches = -1;
  int backward_matches = -1;
  int flags = 0;
  if (STRING_IS_GREEDY_REGEXP(ac_match -> string)) flags |= RE_FLAGS_GREEDY;
  if (STRING_IS_NO_CASE(ac_match -> string)) flags |= RE_FLAGS_NO_CASE;
  if (STRING_IS_DOT_ALL(ac_match -> string)) flags |= RE_FLAGS_DOT_ALL;
  if (STRING_IS_FAST_REGEXP(ac_match -> string)) exec = yr_re_fast_exec;
  else exec = yr_re_exec;
  if (STRING_IS_ASCII(ac_match -> string)) {
    forward_matches = exec(ac_match -> forward_code, data + offset, data_size - offset, offset > 0 ? flags | RE_FLAGS_NOT_AT_START : flags, NULL, NULL);
  }
  if (STRING_IS_WIDE(ac_match -> string) && forward_matches == -1) {
    flags |= RE_FLAGS_WIDE;
    forward_matches = exec(ac_match -> forward_code, data + offset, data_size - offset, offset > 0 ? flags | RE_FLAGS_NOT_AT_START : flags, NULL, NULL);
  }
  switch (forward_matches) {
  case -1:
  case -2:
  case -3:
  case -4:
  case -5:
  }
  if (forward_matches == 0 && ac_match -> backward_code == NULL) callback_args.string = ac_match -> string;
  callback_args.context = context;
  callback_args.data = data;
  callback_args.data_size = data_size;
  callback_args.data_base = data_base;
  callback_args.forward_matches = forward_matches;
  callback_args.full_word = STRING_IS_FULL_WORD(ac_match -> string);
  if (ac_match -> backward_code != NULL) {
    backward_matches = exec(ac_match -> backward_code, data + offset, offset, flags | RE_FLAGS_BACKWARDS | RE_FLAGS_EXHAUSTIVE, _yr_scan_match_callback, (void * ) & callback_args);
    switch (backward_matches) {
    case -2:
    case -3:
    case -4:
    case -5:
    }
  } else {
    FAIL_ON_ERROR(_yr_scan_match_callback(data + offset, 0, flags, & callback_args));
  }
}
        """
    ]
}

# 데이터프레임으로 변환
new_code_df = pd.DataFrame(new_code_data)

# 전처리 함수 재정의
def preprocess_code(code_df):
    # 멀티 라인 주석 제거
    code_df['code'] = code_df['code'].replace(re.compile(r'/\*.*?\*/', re.DOTALL), "", regex=True)
    # 싱글 라인 주석 제거
    code_df['code'] = code_df['code'].replace(re.compile(r'//.*?\n'), "", regex=True)
    # angle brackets를 사용하는 include 제거
    code_df['code'] = code_df['code'].replace(re.compile(r'#include .*?\n'), "", regex=True)
    code_df['code'] = code_df['code'].replace(re.compile(r'#include "*.*?\n'), "", regex=True)
    # 매크로 정의 제거
    code_df['code'] = code_df['code'].replace(re.compile(r'#define .*?\n'), "", regex=True)
    # 전처리 지시문 제거
    code_df['code'] = code_df['code'].replace(re.compile(r'#undef\s+\w+'), "", regex=True)
    code_df['code'] = code_df['code'].replace(re.compile(r'#if\s+\w+'), "", regex=True)
    code_df['code'] = code_df['code'].replace(re.compile(r'#else\s+\w+'), "", regex=True)
    code_df['code'] = code_df['code'].replace(re.compile(r'#elif\s+\w+'), "", regex=True)
    code_df['code'] = code_df['code'].replace(re.compile(r'#endif'), "", regex=True)
    # 탭과 여러 공백을 하나의 공백으로
    code_df['code'] = code_df['code'].replace(re.compile(r'[\t ]+'), " ", regex=True)
    # 여러 줄바꿈을 하나로
    code_df['code'] = code_df['code'].replace(re.compile(r'\n\s*\n'), "", regex=True)
    code_df['code'] = code_df['code'].replace(re.compile(r'\n'), "", regex=True)
    code_df['code'] = code_df['code'].replace(re.compile(r'return*.*?;'), "", regex=True)
    code_df['code'] = code_df['code'].replace(re.compile(r'return;'), "", regex=True)
    return code_df

# 전처리
new_code_df = preprocess_code(new_code_df)

# 모델 및 토크나이저 로드
model_path = '/content/drive/MyDrive/Colab Notebooks/model/codebert(func)'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# 토큰화 함수 재정의
def tokenize_code(code_df):
    return tokenizer(list(code_df['code']), padding='max_length', max_length=512, truncation=True, return_tensors='pt')

# 토큰화
inputs = tokenize_code(new_code_df)

# 모델 예측
with torch.no_grad():
    outputs = model(**inputs)

# 예측 결과 처리
predictions = torch.argmax(outputs.logits, dim=-1)
cwe_ids = [0, 199, 20, 125, 787, 415, 399, 416, 476, 190]  # labels에 대응하는 CWE ID 목록

# 예측된 CWE ID 출력
predicted_cwe_id = cwe_ids[predictions.item()]
print(f'Predicted CWE ID: {predicted_cwe_id}')


Predicted CWE ID: 125


In [ ]:
##### 위에 있는 두 개 통합 #####
import torch
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from IPython.core.display import display, HTML

# 모델과 토크나이저 로드
model_name = 'microsoft/codebert-base-mlm'
model_path = '/content/drive/MyDrive/Colab Notebooks/model/unixcoder(func)'
model = AutoModelForSequenceClassification.from_pretrained(model_path, output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 전처리 함수 재정의
def preprocess_code(code_df):
    code_df['code'] = code_df['code'].replace(re.compile(r'/\*.*?\*/', re.DOTALL), "", regex=True)
    code_df['code'] = code_df['code'].replace(re.compile(r'//.*?\n'), "", regex=True)
    code_df['code'] = code_df['code'].replace(re.compile(r'#include .*?\n'), "", regex=True)
    code_df['code'] = code_df['code'].replace(re.compile(r'#include "*.*?\n'), "", regex=True)
    code_df['code'] = code_df['code'].replace(re.compile(r'#define .*?\n'), "", regex=True)
    code_df['code'] = code_df['code'].replace(re.compile(r'#undef\s+\w+'), "", regex=True)
    code_df['code'] = code_df['code'].replace(re.compile(r'#if\s+\w+'), "", regex=True)
    code_df['code'] = code_df['code'].replace(re.compile(r'#else\s+\w+'), "", regex=True)
    code_df['code'] = code_df['code'].replace(re.compile(r'#elif\s+\w+'), "", regex=True)
    code_df['code'] = code_df['code'].replace(re.compile(r'#endif'), "", regex=True)
    code_df['code'] = code_df['code'].replace(re.compile(r'[\t ]+'), " ", regex=True)
    code_df['code'] = code_df['code'].replace(re.compile(r'\n\s*\n'), "", regex=True)
    code_df['code'] = code_df['code'].replace(re.compile(r'\n'), "", regex=True)
    code_df['code'] = code_df['code'].replace(re.compile(r'return*.*?;'), "", regex=True)
    code_df['code'] = code_df['code'].replace(re.compile(r'return;'), "", regex=True)
    return code_df

# 토큰화 함수 재정의
def tokenize_code(code_df):
    return tokenizer(list(code_df['code']), padding='max_length', max_length=512, truncation=True, return_tensors='pt')

def calculate_attention_per_line(model, tokenizer, code_lines):
    line_attention_scores = []
    for line in code_lines:
        inputs = tokenizer(line, return_tensors='pt', truncation=True, max_length=512, padding=True)
        input_ids = inputs['input_ids']
        attention = model(input_ids, output_attentions=True).attentions
        avg_attn = np.mean([att.detach().cpu().numpy() for att in attention], axis=(0, 1, 2)).squeeze()
        line_attention_scores.append(np.mean(avg_attn))
    return line_attention_scores

def highlight_code_lines(code_lines, attention_scores, threshold=0.01):
    max_attn = max(attention_scores)
    min_attn = min(attention_scores)
    html_code = "<pre>"
    for line, score in zip(code_lines, attention_scores):
        if score > threshold:
            color_intensity = int(255 * (score - min_attn) / (max_attn - min_attn))
            color = f'rgba(255,{255-color_intensity},{255-color_intensity},0.5)'
            html_code += f'<span style="background-color:{color}">{line}</span>\n'
        else:
            html_code += f'{line}\n'
    html_code += "</pre>"
    display(HTML(html_code))

# 새 코드 데이터 준비
new_code_data = {
    'code': [
"""
void func(int type,
  const content::NotificationSource & source,
    const content::NotificationDetails & details) {
  switch (type) {
  case chrome::NOTIFICATION_EXTENSIONS_READY: {
    OnceOffCreateShortcuts();
    break;
  }
  case chrome::NOTIFICATION_EXTENSION_INSTALLED_DEPRECATED: {
    (OS_MACOSX) if (!apps::IsAppShimsEnabled()) break;
    const extensions::InstalledExtensionInfo * installed_info = content::Details <
      const extensions::InstalledExtensionInfo > (details).ptr();
    const Extension * extension = installed_info -> extension;
    if (installed_info -> is_update) {
      web_app::UpdateAllShortcuts(base::UTF8ToUTF16(installed_info -> old_name), profile_, extension);
    } else if (ShouldCreateShortcutFor(profile_, extension)) {
      CreateShortcutsInApplicationsMenu(profile_, extension);
    }
    break;
  }
  case chrome::NOTIFICATION_EXTENSION_UNINSTALLED: {
    const Extension * extension = content::Details <
      const Extension > (details).ptr();
    web_app::DeleteAllShortcuts(profile_, extension);
    break;
  }
  default:
    NOTREACHED();
  }
}
"""
    ]
}

# 데이터프레임으로 변환
new_code_df = pd.DataFrame(new_code_data)

# 전처리
new_code_df = preprocess_code(new_code_df)

# 코드 전처리
preprocessed_code = new_code_df['code'].iloc[0]

# 코드 라인을 분할
code_lines = preprocessed_code.strip().split(';')

# 각 라인의 어텐션 가중치 계산
attention_scores = calculate_attention_per_line(model, tokenizer, code_lines)

# 취약한 라인 강조
highlight_code_lines(code_lines, attention_scores, threshold=0.01)

# 토큰화
inputs = tokenize_code(new_code_df)

# 모델 예측
with torch.no_grad():
    outputs = model(**inputs)

# 예측 결과 처리
predictions = torch.argmax(outputs.logits, dim=-1)
cwe_ids = [0, 199, 20, 125, 787, 415, 399, 416, 476, 190]  # labels에 대응하는 CWE ID 목록

# 예측된 CWE ID 출력
predicted_cwe_id = cwe_ids[predictions.item()]
print(f'Predicted CWE ID: {predicted_cwe_id}')


Predicted CWE ID: 20
